In [1]:
import torch

In [2]:
ckpt = torch.load("output/FastPitch_checkpoint_teacher_100.pt", map_location="cpu")

In [3]:
ckpt["state_dict"].keys()

odict_keys(['pitch_mean', 'pitch_std', 'encoder.word_emb.weight', 'encoder.pos_emb.inv_freq', 'encoder.layers.0.dec_attn.qkv_net.weight', 'encoder.layers.0.dec_attn.qkv_net.bias', 'encoder.layers.0.dec_attn.o_net.weight', 'encoder.layers.0.dec_attn.layer_norm.weight', 'encoder.layers.0.dec_attn.layer_norm.bias', 'encoder.layers.0.pos_ff.CoreNet.0.weight', 'encoder.layers.0.pos_ff.CoreNet.0.bias', 'encoder.layers.0.pos_ff.CoreNet.2.weight', 'encoder.layers.0.pos_ff.CoreNet.2.bias', 'encoder.layers.0.pos_ff.layer_norm.weight', 'encoder.layers.0.pos_ff.layer_norm.bias', 'encoder.layers.1.dec_attn.qkv_net.weight', 'encoder.layers.1.dec_attn.qkv_net.bias', 'encoder.layers.1.dec_attn.o_net.weight', 'encoder.layers.1.dec_attn.layer_norm.weight', 'encoder.layers.1.dec_attn.layer_norm.bias', 'encoder.layers.1.pos_ff.CoreNet.0.weight', 'encoder.layers.1.pos_ff.CoreNet.0.bias', 'encoder.layers.1.pos_ff.CoreNet.2.weight', 'encoder.layers.1.pos_ff.CoreNet.2.bias', 'encoder.layers.1.pos_ff.layer_nor

In [4]:
def check_layer_remove(layer_name):
    if "encoder.layers.1" in layer_name:
        return True
    if "encoder.layers.2" in layer_name:
        return True
    if "encoder.layers.4" in layer_name:
        return True
    if "encoder.layers.5" in layer_name:
        return True

    if "decoder.layers.1" in layer_name:
        return True
    if "decoder.layers.2" in layer_name:
        return True
    if "decoder.layers.4" in layer_name:
        return True
    if "decoder.layers.5" in layer_name:
        return True
    
    return False
    
layer_to_remove = [i for i in ckpt["state_dict"].keys() if check_layer_remove(i)]

In [5]:
layer_to_remove

['encoder.layers.1.dec_attn.qkv_net.weight',
 'encoder.layers.1.dec_attn.qkv_net.bias',
 'encoder.layers.1.dec_attn.o_net.weight',
 'encoder.layers.1.dec_attn.layer_norm.weight',
 'encoder.layers.1.dec_attn.layer_norm.bias',
 'encoder.layers.1.pos_ff.CoreNet.0.weight',
 'encoder.layers.1.pos_ff.CoreNet.0.bias',
 'encoder.layers.1.pos_ff.CoreNet.2.weight',
 'encoder.layers.1.pos_ff.CoreNet.2.bias',
 'encoder.layers.1.pos_ff.layer_norm.weight',
 'encoder.layers.1.pos_ff.layer_norm.bias',
 'encoder.layers.2.dec_attn.qkv_net.weight',
 'encoder.layers.2.dec_attn.qkv_net.bias',
 'encoder.layers.2.dec_attn.o_net.weight',
 'encoder.layers.2.dec_attn.layer_norm.weight',
 'encoder.layers.2.dec_attn.layer_norm.bias',
 'encoder.layers.2.pos_ff.CoreNet.0.weight',
 'encoder.layers.2.pos_ff.CoreNet.0.bias',
 'encoder.layers.2.pos_ff.CoreNet.2.weight',
 'encoder.layers.2.pos_ff.CoreNet.2.bias',
 'encoder.layers.2.pos_ff.layer_norm.weight',
 'encoder.layers.2.pos_ff.layer_norm.bias',
 'encoder.layers.4

In [6]:
import json
f = open("fastpitch_student.json")
student_config = json.load(f)   

In [7]:
for key in layer_to_remove:
    del ckpt["state_dict"][key]

In [8]:
ckpt["state_dict"]["encoder.layers.3.dec_attn.qkv_net.weight"]

tensor([[-0.0283, -0.0030, -0.0215,  ..., -0.0244,  0.0299, -0.0393],
        [ 0.0071, -0.0422,  0.0114,  ..., -0.0229, -0.0753, -0.0458],
        [-0.0087, -0.0292, -0.0314,  ...,  0.0118,  0.0223,  0.0175],
        ...,
        [ 0.0154, -0.0119, -0.0037,  ...,  0.0309,  0.0335, -0.0146],
        [ 0.0124, -0.0238,  0.0190,  ...,  0.0360, -0.0516, -0.0291],
        [ 0.0349,  0.0014, -0.0078,  ..., -0.0241, -0.0313, -0.0046]])

In [32]:
for layer in ckpt["state_dict"].keys():
    if "encoder.layers.3" in layer:
        print(layer)
        new_layer = layer.replace("encoder.layers.3", "encoder.layers.1")
        print(new_layer)
        ckpt["state_dict"][new_layer] = ckpt["state_dict"].pop(layer)
    if "decoder.layers.3" in layer:
        print(layer)
        new_layer = layer.replace("decoder.layers.3", "decoder.layers.1")
        print(new_layer)
        ckpt["state_dict"][new_layer] = ckpt["state_dict"].pop(layer)

In [33]:
len(ckpt["state_dict"].keys())

97

In [34]:
import models

student_model = models.get_model('FastPitch', student_config, device="cpu")

In [35]:
student_model.load_state_dict(ckpt["state_dict"])
student_model.eval()

FastPitch(
  (encoder): FFTransformer(
    (word_emb): Embedding(148, 384, padding_idx=0)
    (pos_emb): PositionalEmbedding()
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-1): 2 x TransformerLayer(
        (dec_attn): MultiHeadAttn(
          (qkv_net): Linear(in_features=384, out_features=192, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
          (dropatt): Dropout(p=0.1, inplace=False)
          (o_net): Linear(in_features=64, out_features=384, bias=False)
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ff): PositionwiseConvFF(
          (CoreNet): Sequential(
            (0): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): ReLU()
            (2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )


In [36]:
len(student_model.state_dict().keys())

97

In [37]:
torch.save({"state_dict": student_model.state_dict()}, "student_100.pt")

In [11]:
student_ckpt = torch.load("test_student.pt")
student_ckpt

FastPitch(
  (encoder): FFTransformer(
    (word_emb): Embedding(148, 384, padding_idx=0)
    (pos_emb): PositionalEmbedding()
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-1): 2 x TransformerLayer(
        (dec_attn): MultiHeadAttn(
          (qkv_net): Linear(in_features=384, out_features=192, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
          (dropatt): Dropout(p=0.1, inplace=False)
          (o_net): Linear(in_features=64, out_features=384, bias=False)
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ff): PositionwiseConvFF(
          (CoreNet): Sequential(
            (0): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): ReLU()
            (2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )


In [38]:
unwrap = lambda m: getattr(m, "module", m)

torch.save({"state_dict": unwrap(student_ckpt).state_dict()}, "test_student_state_dict.pt")


NameError: name 'student_ckpt' is not defined

In [39]:
import re

student_model = models.get_model('FastPitch', student_config, device="cpu")

ckpt = torch.load("student_100.pt")
print(1)

no_pref = lambda sd: {re.sub("^module.", "", k): v for k, v in sd.items()}
unwrap = lambda m: getattr(m, "module", m)

unwrap(student_model).load_state_dict(no_pref(ckpt["state_dict"]))
# student_model.load_state_dict(ckpt["state_dict"])
student_model.eval()

1


FastPitch(
  (encoder): FFTransformer(
    (word_emb): Embedding(148, 384, padding_idx=0)
    (pos_emb): PositionalEmbedding()
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-1): 2 x TransformerLayer(
        (dec_attn): MultiHeadAttn(
          (qkv_net): Linear(in_features=384, out_features=192, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
          (dropatt): Dropout(p=0.1, inplace=False)
          (o_net): Linear(in_features=64, out_features=384, bias=False)
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ff): PositionwiseConvFF(
          (CoreNet): Sequential(
            (0): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): ReLU()
            (2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
